*Liquid smooth and crystal clear... is the hope!*

## Installation: Python environment

With python3.9 the following commands in the command line should create an environment sufficient to run this notebook.

```
    envname="Qmin-vis-env"   
    python3 -m venv "$envname"
    source "$envname"/bin/activate  
    python3 -m pip install pandas pyvistaqt jupyterlab PyQt5 tqdm
    python3 -m ipykernel install --name="$envname" --user
```    

(This assumes `python3` points to an installation of python3.9. I'm not sure about other versions yet but it looks like python3.10 is not working as of 10/27/21. You may have to open jupyter lab as `python3 -m jupyter lab`.)

In [3]:
import os
# os.system('QT_AUTO_SCREEN_SCALE_FACTOR=1')
os.system('QT_SCALE_FACTOR=5')
import numpy as np
from pandas import read_csv
from qtpy import QtWidgets as qw
import qtpy.QtCore as Qt
import pyvista as pv
import pyvistaqt as pvqt


SyntaxError: invalid syntax (2974609661.py, line 8)

Left off: trying to automate adding isosurface and slider (then add to Widgets menu)

In [896]:
class Qmin_plot(pvqt.BackgroundPlotter):
    def __init__(self, filenames=[], user_settings={}):
        super().__init__()
        self.data = []
        self.fullmeshes = []
        self.set_settings(user_settings)
        self.color_defects_by_L24 = False
        self.widgets = {}
        self.slices = {}
        self.isosurfaces = {}
        self.visibility_checkboxes = {}
        self.QSliders_labels = {}
        self.QSliders_updaters = {}        
        self.elements = self.renderer.actors
        pv.global_theme.title = "open-Qmin visualization with pyvistaqt"                
        self.rescale_lights_intensity(16)
        self.renderer.add_axes(interactive=True, color='black') # xyz axes arrows
        self.renderer.set_background("white")              
        self.enable_eye_dome_lighting() 
        self.director_resolution = self.settings["director_resolution"]
        
        if type(filenames) is str:
            filenames = [filenames]
        if len(filenames)>0:
            self.read(filenames) # reads data files into self.data
            self.get_data_from_first_file(self.data[0])            
            for file_data in self.data:
                self.meshdata_from_file(file_data) # sets self.fullmesh and appends it to self.fullmeshes                                               
            self.setup_QSliders()
            self.setup_isosurfaces()
            self.frame_num = 0
            self.load_frame()            
            self.add_mesh(self.fullmesh.outline(), color='black', name="outline") # bounding box)        
            self.setup_director_slice_widget()            
            self.setup_toggle_menus()

        
        
    def scalar_fields(self):
        return [ array_name for array_name in self.fullmesh.array_names 
                 if len(self.fullmesh[array_name].shape)==1 ]
        
    def read(self, filenames, **kwargs):
        for filename in filenames:
            self.data.append(np.array(read_csv(filename, sep="\t", header=None)))
            
    def get_data_from_first_file(self, file_data):
        self.Lx, self.Ly, self.Lz = [ int(item+1) for item in file_data[-1,:3] ]
        self.dims = (self.Lx, self.Ly, self.Lz)
        
    def load_frame(self):
        if self.frame_num >= len(self.fullmeshes):
            self.frame_num = len(self.fullmeshes)-1
        elif self.frame_num < 0:
            self.frame_num = 0
        self.fullmesh = self.fullmeshes[self.frame_num]
        self.make_coarsemesh(self.director_resolution)
        return True
        
    def next_frame(self):
        self.frame_num += 1
        return self.load_frame()            
        
    def previous_frame(self):
        self.frame_num -= 1
        return self.load_frame()            
        
        
    def setup_toggle_menus(self):
        self.toggle_menu = self.main_menu.addMenu('Toggle')
        self.toggle_menu.aboutToShow.connect(self.update_toggle_menu)
#         self.toggle_menu.triggered.connect(self.update_toggle_menu)
#         for actor_name in self.renderer.actors:
#             self.add_to_toggle_menu(actor_name)
        self.the_Add_menu = self.main_menu.addMenu("Add")
        self.widget_menu = self.main_menu.addMenu("Widgets")
        self.widget_menu.aboutToShow.connect(self.update_widget_menu)
        self.the_add_slice_menu = self.the_Add_menu.addMenu("Slice")
        self.the_add_isosurface_menu = self.the_Add_menu.addMenu("Isosurface")
        for array_name in sorted(self.fullmesh.array_names): 
            # only scalar arrays
            if len(self.fullmesh[array_name].shape) == 1:
                menu_action = self.the_add_slice_menu.addAction(
                    array_name, 
                    self.add_slice_aux(array_name)
                )
                menu_action.setCheckable(False)
                menu_action2 = self.the_add_isosurface_menu.addAction(
                    array_name,
                    self.add_isosurface_slider_aux(array_name)
                )
                menu_action2.setCheckable(False)
        self.add_plane_widget_to_widget_menu("director_slice_widget")
        self.widget_menu.triggered.connect(
            lambda: self.widget_menu.actions()[0].setChecked(
                self.widgets["director_slice_widget"].GetEnabled()
            )
        )
        
    def update_toggle_menu(self):
        self.toggle_menu.clear()
        for actor_name in self.renderer.actors.keys():
            # exclude color bars and widget outlines 
            if (not "Addr=" in actor_name 
                and not (len(actor_name.split('outline'))==2
                         and len(actor_name.split('outline')[0])>0
                        )
            ):
                self.add_to_toggle_menu(actor_name)
        for scalar_bar_name in self.scalar_bars.keys():
            self.add_scalar_bar_to_toggle_menu(scalar_bar_name)
            

    def update_widget_menu(self):
        self.widget_menu.clear()
        for widget_name in self.widgets.keys():
            self.add_plane_widget_to_widget_menu(widget_name)
        
    def add_to_toggle_menu(self, actor_name):
        menu_action = self.toggle_menu.addAction(actor_name, 
                              self.generic_menu_toggle(actor_name))
        menu_action.setCheckable(True)
        is_visible = self.renderer.actors[actor_name].GetVisibility()
        menu_action.setChecked(is_visible)
        
    def add_scalar_bar_to_toggle_menu(self, scalar_bar_name):
        scalar_bar = self.scalar_bars[scalar_bar_name]
        menu_action = self.toggle_menu.addAction("└─ " + scalar_bar_name + " colorbar",
            self.toggle_visibility(scalar_bar)
        )
        menu_action.setCheckable(True)
        is_visible = self.scalar_bars[scalar_bar_name].GetVisibility()
        menu_action.setChecked(is_visible)
        

    def toggle_visibility(self, actor):
        def return_function():
            actor.SetVisibility(1-actor.GetVisibility())
        return return_function
        
    def add_plane_widget_to_widget_menu(self, widget_name):
        menu_action = self.widget_menu.addAction(
            widget_name, self.plane_widget_toggle(widget_name))
        menu_action.setCheckable(True)
        menu_action.setChecked(self.interactor.widgets[widget_name].GetEnabled())
        
        
    def toggle_color_by_L24(self):
        self.color_defects_by_L24 = 1 - self.color_defects_by_L24
        slider = self.QSliders['defects']
        for delta in [1, -1]:
            slider.setValue(slider.value()+delta)
        
        
    def setup_isosurfaces(self):
        self.setup_boundaries()
        
        self.QSliders_toolbar.addSeparator()
        self.defects_label, self.set_defect_S, self.QSliders["defects"] = self.add_QSlider(
            self.update_defects,
            self.QSliders_toolbar,
            "defects",            
            scalars=self.fullmesh["order"],
            min_val=0,
            max_val=1,
            init_val=30,
            label_txt="S"
        )

        
        
        self.L24_color_button = qw.QCheckBox("L24 color")
        self.L24_color_button.toggled.connect(self.toggle_color_by_L24)
        self.QSliders_toolbar.addWidget(self.L24_color_button)
        self.update_defects(0.3)         # initialize the actor
        
        self.add_viscolor_toolbar("defects")
#         self.add_color_picker_button("defects")

#         self.add_visibility_checkbox("defects")
        

        
        
    def setup_QSliders(self):
        self.QSliders_toolbar = qw.QToolBar('QSliders')
        self.QSliders_toolbar.setFixedWidth(150)
        self.app_window.addToolBar(Qt.Qt.LeftToolBarArea, self.QSliders_toolbar)
        self.QSliders={}
        self.lighting_slider_label = qw.QLabel()
        self.lighting_slider = qw.QSlider(Qt.Qt.Horizontal)
        self.lighting_slider.setMinimum(0)
        self.lighting_slider.setMaximum(50)
        self.lighting_slider.setValue(9)
        self.QSliders_toolbar.addWidget(self.lighting_slider_label)
        self.QSliders_toolbar.addWidget(self.lighting_slider)        
        self.lighting_slider.valueChanged.connect(self.set_lights_intensity)
        self.lighting_slider.setValue(8)
        
        
        self.n_res_slider_label = qw.QLabel()
        self.n_res_slider = qw.QSlider(Qt.Qt.Horizontal)
        self.n_res_slider.setMinimum(1)
        self.n_res_slider.setMaximum(
            int(max(1,max(np.array([self.Lx,self.Ly,self.Lz])/6)))
        )
        self.n_res_slider.setValue(self.director_resolution)        
        self.QSliders_toolbar.addWidget(self.n_res_slider_label)    
        self.QSliders_toolbar.addWidget(self.n_res_slider)
        self.n_res_slider.valueChanged.connect(self.make_coarsemesh)            
            
    def setup_boundaries(self):
        boundary_vis_kwargs = {
            "pbr":True,
            "metallic":1,
            "roughness":0.5,
            "diffuse":1,
            "smooth_shading":True
        }
        self.isosurfaces["boundaries"] = self.add_mesh(
            self.fullmesh.contour([0.5], scalars="nematic_sites"), 
            show_scalar_bar=False, 
            color=self.settings["boundaries_color"],
            name="boundaries",
            #pbr=True, metallic=1, roughness=0.5, diffuse=1, smooth_shading=True
            **boundary_vis_kwargs
        )
        
        for i in range(1,self.num_boundaries+1):
            bdy = f"boundary_{i}"
            self.isosurfaces[bdy] = self.add_mesh(
                self.fullmesh.contour([0.5], scalars=bdy),
                show_scalar_bar=False,
                color=self.settings["boundaries_color"],
                name=bdy,
                **boundary_vis_kwargs
            )
            self.renderer.actors[bdy].SetVisibility(0)
            
        
    def setup_director_slice_widget(self):
        widget_name="director_slice_widget"
        if widget_name in self.widgets.keys():
            widget = self.widgets[widget_name]
            normal=widget.GetNormal()
            origin=widget.GetOrigin()
            enabled=widget.GetEnabled()
            widget.SetEnabled(0)            
        else:
            normal=(1.0, 0.0, 0.0)
            origin=(self.Lx/2, self.Ly/2, self.Lz/2)
            enabled=True
            
        self.widgets[widget_name] = self.add_plane_widget(
            self.director_slice_func,
            factor=1.1, 
            color=self.settings["plane_widget_color"],
            tubing=True,
            normal=normal,
            origin=origin
        )
        self.widgets[widget_name].SetEnabled(enabled)
        
        
    def Q33_from_Q5(self, Q5):
        (Qxx, Qxy, Qxz, Qyy, Qyz) = Q5.T
        Qmat = np.moveaxis(np.array([ 
                [Qxx, Qxy, Qxz],
                [Qxy, Qyy, Qyz],
                [Qxz, Qyz, -Qxx-Qyy]
                ]), -1, 0)
        return Qmat

    def n_from_Q(self, Qmat):
        """Get director from 3x3-matrix Q-tensor data"""
        evals, evecs = np.linalg.eigh(Qmat)    
        return evecs[:,:,2]            

    def set_settings(self, user_settings):
        self.settings = {
            "boundaries_color":"gray",
            "director_color":"red",
            "director_resolution":2, 
            "default_defect_S":0.3, # initialization order value for defect isosurfaces
            "defects_color":(37/256,150/256,190/256),
            "checkbox_size":50, # size of toggle boxes in pixels
            "checkbox_spacing":10, # spacing between toggle boxes in pixels
            "window_size":(1200,800), # window size in pixels
            "cylinder_resolution":8, # angular resolution for each cylindrical rod; larger values look nicer but take longer to compute
            "slice_plane_color":"lightyellow", # set to None to use slice_color_function instead    
            "slice_cmap":"cividis", # color map for use with slice_color_function
            "slice_color_function":(lambda slc: np.abs(slc["director"][:,0])), # optionally color slice plane by some function of director or order
            "plane_widget_color":"orange",
            "default_isosurface_color":"purple"
        }
        for key, value in zip(user_settings.keys(), user_settings.values()):
            self.settings[key] = value

            
    def meshdata_from_file(self, dat):

        # name the data columns:
        self.coords = dat[:,:3]
        self.Qdata = dat[:,3:8]
        self.Q33 = self.Q33_from_Q5(self.Qdata) # 3x3 Q matrix
        self.site_types = dat[:,8]
        self.order = dat[:,9]

        # grid for pyvista:
        self.fullmesh = pv.UniformGrid((self.Lx,self.Ly,self.Lz)) 
        # Order (defects) data:
        self.order[self.site_types>0] = np.max(self.order) # Don't plot defects inside objects
        self.fullmesh["order"] = self.order

        # director data:
        self.Qdata[self.site_types>0] = 0. # Don't bother calculating eigenvectors inside objects
        self.fullmesh["director"] = self.n_from_Q(self.Q33)

        # boundaries:
        self.fullmesh["nematic_sites"] = 1*(self.site_types<=0)
        self.num_boundaries = int(np.max(self.site_types))
        for i in range(1,self.num_boundaries+1):
            self.fullmesh[f"boundary_{i}"] = 1*(self.site_types==i)

        
        self.Q33_xyz = self.Q33.reshape((self.Lx,self.Ly,self.Lz,3,3))        
        self.diQjk = np.moveaxis(
            np.array(
                [np.roll(self.Q33_xyz,-1, axis=i) 
                 - np.roll(self.Q33_xyz,1, axis=i) 
                 for i in range(3)]
            ),
            0, -3
        )
        
        # energy:
        self.fullmesh["energy_L1"] = np.sum(
            self.diQjk**2, axis=(-1,-2,-3)).flatten()
        self.fullmesh["energy_L2"] = np.sum(
            (np.einsum("...iij->...j", self.diQjk))**2, 
            axis=-1
        ).flatten() # djQij dkQik = djQji dkQki = sum_i[(djQji)^2]
        self.fullmesh["energy_L6"] = np.einsum(
            "...ij,...ikl,...jkl", self.Q33_xyz, self.diQjk, self.diQjk).flatten()
        self.fullmesh["energy_L3"] = np.einsum(
            "...ijk,...kij", self.diQjk, self.diQjk).flatten() 
        self.fullmesh["energy_L24"] = self.fullmesh["energy_L3"] - self.fullmesh["energy_L2"] # diQjk dkQij - diQij dkQjk 
        for i in [1,2,3,6,24]:
            self.fullmesh[f"energy_L{i}"] *= 1*(self.site_types==0)        
        L1 = self.fullmesh["energy_L1"]
        L2 = self.fullmesh["energy_L2"]
        L3 = self.fullmesh["energy_L3"]
        L6 = self.fullmesh["energy_L6"]
        S = self.fullmesh["order"]
        # TODO: account for q0 
        self.fullmesh["energy_K1"] = 2/(9*S*S) * (-L1/3 + 2*L2 -2/(3*S)*L6)
        self.fullmesh["energy_K2"] = 2/(9*S*S) * (L1 - 2*L3)
        self.fullmesh["energy_K3"] = 2/(9*S*S) * (L1/3 + 2/(3*S)*L6)        
        self.fullmeshes.append(self.fullmesh)   
        for i, dataset_name in enumerate(["|n_x|", "|n_y|", "|n_z|"]):
            self.fullmesh[dataset_name] = np.abs(self.fullmesh["director"][...,i])
        
      
    def make_coarsemesh(self, nres):
        self.director_resolution = nres
        self.coarsemesh = self.fullmesh.probe(
            pv.UniformGrid(tuple([ int(item/nres) for item in self.dims]),
                           (nres,)*3
                          )
        )   
        try:
            self.n_res_slider_label.setText(f"n_res: {nres}")                                        
        except AttributeError:
            pass
        
        self.director_slice_func = self.make_director_slice_func()
        
    
    def plane_widget_toggle(self, widget_name):
        def return_function():
            if widget_name in self.widgets.keys():
                widget = self.widgets[widget_name]
                if widget.GetEnabled():
                    widget.EnabledOff()
                else:
                    widget.EnabledOn()             
        return return_function

    def make_director_slice_func(self):        
        def director_slice_func(normal, origin):
            """make glyph plot and transparent plane for director field slice"""
            
            origin = tuple(
                self.director_resolution * np.asarray(
                    np.array(origin)/self.director_resolution, dtype=int
                )
            )
            slc = self.coarsemesh.slice(normal=normal, origin=origin)
            cylinders = slc.glyph(orient="director", scale="nematic_sites",
                factor=self.director_resolution,
                geom=pv.Cylinder(
                    radius=0.2, height=1, 
                    resolution=self.settings["cylinder_resolution"]
                ),  
                tolerance=None
    #             progress_bar=True
            )
            try:
                director_vis = self.renderer.actors["director"].GetVisibility()                
                slice_plane_vis = self.renderer.actors["slice_plane"].GetVisibility()
            except KeyError:
                director_vis = 1
                slice_plane_vis = 1
            self.renderer.actors["director"] = self.add_mesh(
                cylinders, 
                color=self.settings["director_color"], 
    #             specular=1, ambient=0.35, 
                pbr=True, metallic=0.5, roughness=0.25, diffuse=1,
                name="director" # "name" == actor's name so old actor is replaced
            )
            self.renderer.actors["director"].SetVisibility(director_vis)

            self.renderer.actors["slice_plane"] = self.add_mesh(
                slc, opacity=0.01, 
                ambient=1, diffuse=0, specular=0, # glows, doesn't reflect
                color=self.settings["slice_plane_color"], 
                scalars=(self.settings["slice_color_function"](slc) 
                    if self.settings["slice_plane_color"] is None 
                    else None # use slice_color_function only if slice_plane_color==None
                ),
                cmap=self.settings["slice_cmap"],
                name="slice_plane" # "name" == actor's name so old actor is replaced
            )
            self.renderer.actors["slice_plane"].SetVisibility(slice_plane_vis)
#             except ValueError:
#                 pass
            
        return director_slice_func
      
        
    def generic_slider_callback(self, actor_name, scalars, contour_value, 
        color=None, mesh_color_scalars=None, clim=None, cmap=None):
        """generic callback function for slider controlling isosurfaces"""
        kwargs={
            "show_scalar_bar":False,
#             "specular":1, "specular_power":20,
#             "ambient":0.5, "diffuse":1,
            "smooth_shading":True,
            "name":actor_name,
            "pbr":True, 
            "metallic":0.5,
            "roughness":0.25,
            "diffuse":1,   
            "name":actor_name
        }
        if mesh_color_scalars is not None:
            kwargs["scalars"] = mesh_color_scalars
            if clim is not None:
                kwargs["clim"] = clim
            if cmap is not None:
                kwargs["cmap"] = cmap            
        elif color is not None:
            kwargs["color"] = color
        elif actor_name+"_color" in self.settings.keys():
            kwargs["color"] = self.settings[actor_name+"_color"]
        else:
            kwargs["color"] = self.settings["default_isosurface_color"]
        
        try:
            self.renderer.actors[actor_name] = self.add_mesh(
                self.fullmesh.contour(
                    [contour_value], scalars=scalars
                ), **kwargs            
            ) 
            actor = self.renderer.actors[actor_name]
            if actor_name in self.visibility_checkboxes.keys():
                self.visibility_checkboxes[actor_name].toggled.connect(
                    lambda : actor.SetVisibility(1 - actor.GetVisibility())
                )
        except ValueError: # ignore "empty mesh" warnings 
            pass
        
     
    def update_defects(self, dthresh):
        return self.generic_slider_callback(
            "defects", "order", dthresh, 
            color=self.settings["defects_color"] if not self.color_defects_by_L24 else None, 
            mesh_color_scalars="energy_L24" if self.color_defects_by_L24 else None, 
            clim=[-1,1], cmap="jet"
        )
    
    
##     the sliders

    def add_isosurface_slider(self, dataset_name, actor_name=None, mesh=None, color=None, label_txt=None, min_val=None, max_val=None):
        if actor_name is None:
            actor_name = dataset_name+"_isosurface"
        while actor_name in self.renderer.actors.keys():
            actor_name += "\'"        

        if mesh is None:
            mesh = self.fullmesh
        if label_txt is None:
            label_txt = dataset_name
        dataset=mesh[dataset_name]
        if min_val is None:
            min_val = np.min(dataset)
        if max_val is None:
            max_val = np.max(dataset)
                
        self.QSliders_toolbar.addSeparator()
        self.QSliders_labels[actor_name], self.QSliders_updaters[actor_name], self.QSliders[actor_name] = self.add_QSlider(
            lambda value: self.generic_slider_callback(
               actor_name, dataset_name, value, color
            ),
            self.QSliders_toolbar,
            actor_name,
            scalars=mesh[dataset_name],
            label_txt=label_txt,
            min_val = min_val,
            max_val = max_val
        )
        self.QSliders[actor_name].setValue(51)
        self.QSliders[actor_name].setValue(50)
        self.renderer.actors[actor_name].SetVisibility(1) 
        self.add_viscolor_toolbar(actor_name)
        
    def add_viscolor_toolbar(self, actor_name):
        toolbar_row = qw.QToolBar()
        self.add_color_picker_button(actor_name, toolbar_row)
        self.add_visibility_checkbox(actor_name, toolbar_row)
        self.QSliders_toolbar.addWidget(toolbar_row)
        

        
    def add_QSlider(self, update_method, toolbar, actor_name, num_divs=100, 
                    init_val=50, min_val = None, max_val = None, scalars=None, label_txt=None):
        def slider_formula(slider_value):
            return min_val + (max_val-min_val)*slider_value/100         
        def external_update(float_value):
            slider_value = int( 100*(float_value-min_val)/(max_val-min_val))
            slider.setValue(slider_value)
                
        slider = qw.QSlider(Qt.Qt.Horizontal)
        slider.setMinimum(0)
        slider.setMaximum(num_divs)
        slider.setValue(init_val)
        
        text_row = qw.QToolBar()
        if label_txt is not None:
            label = qw.QLabel(label_txt)
            text_row.addWidget(label)
#             label = qw.QLabel(f"{label_txt}: {slider_formula(slider.value()):.3f}")
#             toolbar.addWidget(label)
#         else:
#             label = None
        
        spinbox = qw.QDoubleSpinBox()
        spinbox.setDecimals(3)
        spinbox.setSingleStep(0.1)
        spinbox.setValue(slider_formula(init_val))
        spinbox.valueChanged.connect(external_update)        
        text_row.addWidget(spinbox)
        toolbar.addWidget(text_row)
        

        if max_val is None and scalars is not None:
            max_val = np.max(scalars)
        if min_val is None and scalars is not None:
            min_val = np.min(scalars)

        def valuechange_method(slider_value):                        
            float_value = slider_formula(slider_value)
            if actor_name in self.renderer.actors:
                vis = self.renderer.actors[actor_name].GetVisibility()
            else:
                vis = 0
            update_method(float_value)
            self.renderer.actors[actor_name].SetVisibility(vis)
            spinbox.setValue(float_value)
#             if label is not None:
#                 label.setText(f"{label_txt}: {float_value:.3f}")
        slider.valueChanged.connect(valuechange_method)            
        toolbar.addWidget(slider)
        return label, external_update, slider

    def generic_menu_toggle(self, actor_name):        
        def return_function():
            actor=self.renderer.actors[actor_name]    
            if actor_name in self.visibility_checkboxes.keys():
                self.visibility_checkboxes[actor_name].setChecked(1-actor.GetVisibility())
            else:
                actor.SetVisibility(1-actor.GetVisibility())
                if actor_name in self.QSliders.keys():
                    slider = self.QSliders[actor_name]
                    slider.setValue(slider.value()+1)
                    slider.setValue(slider.value()-1)
            if actor_name in self.scalar_bars:
                self.scalar_bars[actor_name].SetVisibility(actor.GetVisibility())
        return return_function

    def set_lights_intensity(self, intensity):
        for light in self.renderer.lights:
            light.SetIntensity(intensity)
        self.lighting_slider_label.setText(f"lighting: {intensity}")
    
    def rescale_lights_intensity(self, factor):
        for light in self.renderer.lights:
            light.SetIntensity(factor*light.GetIntensity())
    
    def add_isosurface_slider_aux(self, scalars_name):
        def return_function():
            self.add_isosurface_slider(scalars_name)
        return return_function

    
    def add_slice_aux(self, scalars_name):
        def return_function():
            self.add_slice(scalars_name)
        return return_function
    
    def add_slice(self, scalars_name, slice_name=None, widget_name=None):
        if slice_name is None:
            slice_name = scalars_name+"_slice"
            while slice_name in self.elements.keys():
                slice_name += "\'"
        if widget_name is None:
            widget_name = scalars_name + "_widget"
            while widget_name in self.widgets.keys():
                widget_name += "\'"
        
        self.slices[slice_name] = self.add_mesh_slice(self.fullmesh, 
            scalars=scalars_name, name=slice_name, 
            ambient=1, specular=0, diffuse=0, 
            cmap='cividis',
            clim=(lambda arr: 
                  [ np.average(arr) - ((-1)**i)*2*np.std(arr) for i in range(2) ]
            )(self.fullmesh[scalars_name]),
            scalar_bar_args={"interactive":True,
                             #"height":0.25, "width":0.05, 
                             "vertical":True,
                            "title_font_size":10, "color":"black", 
                             "label_font_size":8, "shadow":False,
                            "n_labels":3,
                            "font_family":"arial"
                            }
        )
        scalar_bar = self.scalar_bars[scalars_name]
        scalar_bar.SetMaximumHeightInPixels(200)
        scalar_bar.SetHeight(50)
        scalar_bar.SetMaximumWidthInPixels(50)
        
        self.elements[slice_name].SetVisibility(True)        
        self.add_to_toggle_menu(slice_name)
        self.widgets[widget_name] = self.plane_widgets[-1]
        self.add_plane_widget_to_widget_menu(widget_name)        
        
        
    def color_picker(self, actor_name):
        self.settings[actor_name + "_color"] = qw.QColorDialog.getColor().name()
        if actor_name in self.QSliders.keys():
            try: 
                for i in [1,-1]:
                    self.QSliders[actor_name].setValue(self.QSliders[actor_name].value()+i)
            except IndexError:
                for i in [-1,1]:
                    self.QSliders[actor_name].setValue(self.QSliders[actor_name].value()+i)
                    
    def add_color_picker_button(self, actor_name, toolbar):
        color_button = qw.QPushButton('🎨')#'color')
#         color_button.setFixedWidth(50)
        color_button.setToolTip('choose color for '+actor_name)
        color_button.clicked.connect(lambda: self.color_picker(actor_name))
        toolbar.addWidget(color_button)
        
    def add_visibility_checkbox(self, actor_name, toolbar):
        checkbox = qw.QCheckBox('👁')#'show')
        checkbox.setToolTip('toggle visibility of '+actor_name)
        actor=self.renderer.actors[actor_name]
        checkbox.setChecked(actor.GetVisibility())        
        checkbox.toggled.connect(lambda : actor.SetVisibility(1-actor.GetVisibility()))
        toolbar.addWidget(checkbox)
        self.visibility_checkboxes[actor_name] = checkbox
        
        

In [897]:
# pv.close_all()
qp = Qmin_plot("tetstGUIbdys_x0y0z0.txt")

In [891]:
qp.scalar_bars["order"].SetMaximumHeightInPixels(200)
qp.scalar_bars["order"].SetMaximumWidthInPixels(50)
qp.scalar_bars["|n_y|"].SetHeight(200)

In [902]:
qp.interactor.scalar_bars

Scalar Bar Title     Interactive

In [823]:

actor_name = 'energy_K2_isosurface'
self=qp

spinbox = qw.QDoubleSpinBox()
spinbox.setDecimals(3)
spinbox.valueChanged.connect(
    lambda value: self.QSliders_updaters[actor_name](value))
self.QSliders_toolbar.addWidget(spinbox)

In [844]:
dir(qw.QSpinBox)

['AdaptiveDecimalStepType',
 'ButtonSymbols',
 'CorrectToNearestValue',
 'CorrectToPreviousValue',
 'CorrectionMode',
 'DefaultStepType',
 'DrawChildren',
 'DrawWindowBackground',
 'IgnoreMask',
 'NoButtons',
 'PaintDeviceMetric',
 'PdmDepth',
 'PdmDevicePixelRatio',
 'PdmDevicePixelRatioScaled',
 'PdmDpiX',
 'PdmDpiY',
 'PdmHeight',
 'PdmHeightMM',
 'PdmNumColors',
 'PdmPhysicalDpiX',
 'PdmPhysicalDpiY',
 'PdmWidth',
 'PdmWidthMM',
 'PlusMinus',
 'RenderFlag',
 'RenderFlags',
 'StepDownEnabled',
 'StepEnabled',
 'StepEnabledFlag',
 'StepNone',
 'StepType',
 'StepUpEnabled',
 'UpDownArrows',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'acceptDrops',

In [776]:
little_row = qw.QToolBar()
checkbox1 = qw.QCheckBox('show')
checkbox2 = qw.QCheckBox('show')
little_row.addWidget(checkbox1)
little_row.addWidget(checkbox2)
qp.QSliders_toolbar.addWidget(little_row)

In [701]:
qp.add_visibility_checkbox('energy_K3_isosurface')visibility_checkboxes

In [672]:
qp.widget_menu.actions()[0].setObjectName('hi')

In [705]:
qp.renderer.actors['energy_K2_isosurface'].SetVisibility(0)

In [739]:
qp.renderer.actors

{'boundaries': (vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor)0x7f88214ece80,
 'boundary_1': (vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor)0x7f88214ec700,
 'boundary_2': (vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor)0x7f88214ecdc0,
 'defects': (vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor)0x7f88214ec9a0,
 'outline': (vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor)0x7f88214fa160,
 'director': (vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor)0x7f88214fa580,
 'slice_plane': (vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor)0x7f88214faa60,
 'energy_K3_isosurface': (vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor)0x7f88214da9a0,
 "energy_K3_isosurface'": (vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor)0x7f8821536a00}

In [653]:
checkbox.toggled.

In [624]:
qw.QCheckBox().clicked.connect()

TypeError: connect() missing required argument 'slot' (pos 1)

In [ ]:
checkbox.

In [587]:
color_button = qw.QPushButton('color')
color_button.clicked.connect(lambda: qp.color_picker('defects'))
qp.QSliders_toolbar.addWidget(color_button)

Traceback (most recent call last):
  File "/var/folders/pt/1x9c1p2s02ggqr8yh1p00ln40000gn/T/ipykernel_46173/1073132292.py", line 93, in <lambda>
    lambda: qp.widget_menu.actions()[1].setChecked(
IndexError: list index out of range


In [304]:
qp.checkboxes_toolbar = qw.QToolBar()
qp.color_defects_by_L24_button = qw.QCheckBox("L24 color")

def toggle_color_by_L24():
    qp.color_defects_by_L24 = 1 - qp.color_defects_by_L24
    slider = qp.QSliders['defects']
    slider.setValue(slider.value()+1)
    slider.setValue(slider.value()-1)
#     qp.QSliders['defects'].setValue(qp.QSliders['defects'].value()+1)
#     qp.QSliders['defects'].setValue(qp.QSliders['defects'].value()-1)
    
qp.color_defects_by_L24_button.toggled.connect(toggle_color_by_L24)
qp.checkboxes_toolbar.addWidget(qp.color_defects_by_L24_button)
qp.app_window.addToolBar(Qt.Qt.TopToolBarArea, qp.checkboxes_toolbar)

TypeError: SetShaderProperty argument 1: method requires a VTK object

In [224]:
qp.QSliders['defects'].updatesEnabled()
qp.QSliders['defects'].setValue(qp.QSliders['defects'].value()+1)

In [210]:
qp.renderer.enable_parallel_projection()

###### import qtpy.QtGui as qg

def blah(event):
    qp.lighting_slider_label.setText('aoweifj')

qp.lighting_slider_label.mousePressEvent = blah


In [15]:
# b = qw.QLineEdit("fish")
# qp.QSliders_toolbar.addWidget(b)
b.setText("moo")

def rename_window(newname):
    qp.app_window.setWindowTitle(newname)
    
b.textEdited.connect(rename_window)

In [25]:
c = qw.QDoubleSpinBox()
c.setPrefix("S  ")
c.setRange(-2.,5.)
c.setDecimals(4)
c.setSingleStep(0.1)
c.setValue(2)
qp.QSliders_toolbar.addWidget(c)

In [ ]:
qw.QPushButton

In [3]:
qp = Qmin_plot(["tetstGUIbdys_x0y0z0.txt"])

In [74]:
qp.next_frame_button=qw.QToolButton()

In [78]:
qp.next_frame_button.addAction(qp, qp.next_frame())

TypeError: addAction(self, QAction): argument 1 has unexpected type 'Qmin_plot'

In [ ]:
qp.app_window.setMouseTracking(True)
qp.app_window.mouseMoveEvent(qp.pos())

In [37]:
help(qp.app_window.mouseMoveEvent())

TypeError: mouseMoveEvent(self, QMouseEvent): not enough arguments

In [28]:
qp.frame_toolbar = qw.QToolBar('frames')
qp.app_window.addToolBar(qp.frame_toolbar)
qp.next_frame_button = qw.QPushButton(">")

In [106]:
self=qp
n_xyz = qp.fullmesh["director"].reshape(self.Lx, self.Ly, self.Lz, 3)
dinj = np.moveaxis(
    np.array(
    [np.roll(n_xyz, -1, axis=i) - np.roll(n_xyz, 1, axis=i) 
     for i in range(3)]),
    0, -2
)
#     self.Q33_xyz = self.Q33.reshape((self.Lx,self.Ly,self.Lz,3,3))        
#         self.diQjk = np.moveaxis(
#             np.array(
#                 [np.roll(self.Q33_xyz,-1, axis=i) 
#                  - np.roll(self.Q33_xyz,1, axis=i) 
#                  for i in range(3)]
#             ),
#             0, -3
#         ) 
#     

In [115]:
dinj = np.empty(qp.dims + (3,3))
for i in range(3):
    n_i_up = np.roll(n_xyz, -1, axis=i)
    n_i_dn = np.roll(n_xyz, 1, axis=i)
    n_i_dn *= np.stack( (np.sign(np.sum(n_i_up * n_i_dn, axis=-1)),)*3, axis=-1)
    dinj[...,i,:] = n_i_up - n_i_dn
dinj.shape    

(50, 50, 50, 3, 3)

In [118]:
div_n = np.einsum("...,i,i->...,i", dinj)

ValueError: more operands provided to einstein sum function than specified in the subscripts string

In [ ]:
qp.next_frame_button.addAction(Qt.QA)

In [160]:

qp.scalar_bars["energy_K2"].SetPosition([0])
qp.scalar_bars["energy_K2"].SetDragable(1)

# qp.scalar_bars["energy_K2"].keys()setup_director_slice_widget

In [465]:
qp.scalar_bars

Scalar Bar Title     Interactive
"energy_L1"          False

In [463]:
qp.renderer.actors['Addr=0x7fdd160190b0'].SetVisibility(1)
qp.renderer.actors['Addr=0x7fdd160190b0'].SetPickable(1)
qp.renderer.actors['Addr=0x7fdd160190b0'].SetDragable(1)

KeyError: 'Addr=0x7fdd160190b0'